In [48]:
import streamlit as st
import pandas as pd
import plotly.express as px

from mainnet_launch.constants import AutopoolConstants, CACHE_TIME, eth_client
from mainnet_launch.constants import BAL_ETH, AUTO_ETH, AUTO_LRT
from mainnet_launch.lens_contract import get_pools_and_destinations_call, build_proxyGetDestinationSummaryStats_call
from mainnet_launch.data_fetching.get_state_by_block import get_raw_state_by_blocks, get_state_by_one_block
from mainnet_launch.destinations import DestinationDetails, get_destination_details
from mainnet_launch.data_fetching.get_state_by_block import build_blocks_to_use


from mainnet_launch.destinations import DestinationDetails, get_destination_details

destinations = [d for d in get_destination_details() if d.autopool.name == 'balETH']




2024-11-07 14:32:46.657 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-07 14:32:46.658 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-07 14:32:46.659 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-07 14:32:46.660 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-07 14:32:46.660 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [ ]:
calls = []
for i, d in enumerate(destinations):
    name = d.vault_name + ' ' + str(i)
    call = build_proxyGetDestinationSummaryStats_call(
        name, BAL_ETH.autopool_eth_strategy_addr, d.vaultAddress, 'out', 1
    )
    calls.append(call)



# first_few_blocks_with_rebalances = [20765628, 20769006, 20774007]
blocks = build_blocks_to_use()
df = get_raw_state_by_blocks(calls, blocks)

pool_and_destination_df =  get_raw_state_by_blocks([get_pools_and_destinations_call()], blocks)





2024-11-07 14:31:20.948 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-07 14:31:20.948 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-07 14:31:20.949 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-07 14:31:20.950 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-07 14:31:20.950 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [66]:
balETH_nav_per_share = pool_and_destination_df.map(lambda cell: cell['autopools'][1]['navPerShare']/1e18)
balETH_tvl_totalAssets = pool_and_destination_df.map(lambda cell: cell['autopools'][1]['totalAssets']/1e18)
balETH_tvl_totalSupply = pool_and_destination_df.map(lambda cell: cell['autopools'][1]['totalSupply']/1e18)

pricePerShare = df.map(lambda cell: cell["pricePerShare"] if isinstance(cell, dict) else None).astype(float)
ownedShares = df.map(lambda cell: cell["ownedShares"] if isinstance(cell, dict) else None).astype(float)
priceReturn = df.map(lambda cell: cell["priceReturn"] if isinstance(cell, dict) else None).astype(float)
destination = df.map(lambda cell: cell["destination"] if isinstance(cell, dict) else None)
tvl = (pricePerShare * ownedShares).sum(axis=1)
scaled_tvl =( (pricePerShare / (1-priceReturn)) * ownedShares).sum(axis=1)
scaled_nav_per_share = (scaled_tvl / balETH_tvl_totalSupply['getPoolsAndDestinations'])
scaled_nav_per_share

timestamp
2024-09-15 02:04:47+00:00    1.000000
2024-09-15 05:44:35+00:00    1.000000
2024-09-15 09:24:47+00:00    1.000000
2024-09-15 13:04:11+00:00    1.000000
2024-09-15 16:44:23+00:00    1.000000
                               ...   
2024-11-07 02:45:35+00:00    1.004844
2024-11-07 06:25:23+00:00    1.004842
2024-11-07 10:05:23+00:00    1.004870
2024-11-07 13:45:47+00:00    1.004824
2024-11-07 17:27:11+00:00    1.004841
Length: 352, dtype: float64

In [74]:
tvl_df = pd.DataFrame()
tvl_df['totalAssets'] = balETH_tvl_totalAssets
tvl_df['tvl'] = tvl
tvl_df['scaled_tvl'] = scaled_tvl
px.line(tvl_df/ balETH_tvl_totalSupply.values)

In [75]:
priceReturn

,weETH/ezETH/rswETH (balancer) 0,B-rETH-STABLE (balancer) 1,weETH/ezETH/rswETH (balancer) 2,ezETH-WETH-BPT (balancer) 3,pxETH/wETH (balancer) 4,weETH/rETH (balancer) 5,rsETH / ETHx (balancer) 6,osETH/wETH-BPT (balancer) 7,osETH/wETH-BPT (balancer) 8,weETH/rETH (balancer) 9,...,rsETH / WETH (balancer) 27,ECLP-wstETH-cbETH (balancer) 28,ETHx/wstETH (balancer) 29,weETH/ezETH/rswETH (balancer) 30,ECLP-wstETH-cbETH (balancer) 31,pxETH/wETH (balancer) 32,pxETH/wETH (balancer) 33,B-rETH-STABLE (balancer) 34,ETHx/wstETH (balancer) 35,wstETH-WETH-BPT (balancer) 36
timestamp,,,,,,,,,,,,,,,,,,,,,
2024-09-15 02:04:47+00:00,0.000797,NaN,NaN,NaN,NaN,-0.000265,NaN,-0.000821,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.000647,NaN,NaN,NaN,NaN
2024-09-15 05:44:35+00:00,0.000797,NaN,NaN,NaN,NaN,-0.000275,NaN,-0.000816,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.000647,NaN,NaN,NaN,NaN
2024-09-15 09:24:47+00:00,0.000780,NaN,NaN,NaN,NaN,-0.000322,NaN,-0.000836,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.000647,NaN,NaN,NaN,NaN
2024-09-15 13:04:11+00:00,0.000782,NaN,NaN,NaN,NaN,-0.000296,NaN,-0.000831,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.000647,NaN,NaN,NaN,NaN
2024-09-15 16:44:23+00:00,0.000782,NaN,0.000782,NaN,NaN,-0.000302,0.000855,-0.000826,-0.000826,NaN,...,NaN,NaN,NaN,NaN,0.001504,-0.001237,-0.001237,-0.000645,0.000634,0.000195
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-11-07 02:45:35+00:00,0.002036,NaN,0.002036,0.000340,0.001485,0.001475,0.000000,NaN,NaN,0.001475,...,0.000683,NaN,0.000431,0.002036,NaN,0.001485,0.000000,NaN,0.000000,0.000064
2024-11-07 06:25:23+00:00,0.002037,NaN,0.002037,0.000340,0.001500,0.001513,0.000000,NaN,NaN,0.001513,...,0.000680,NaN,0.000431,0.002037,NaN,0.001500,0.000000,NaN,0.000000,0.000061
2024-11-07 10:05:23+00:00,0.002000,NaN,0.002000,0.000385,0.001426,0.001254,0.000000,NaN,NaN,0.001254,...,0.000614,NaN,0.000385,0.002000,NaN,0.001426,0.000000,NaN,0.000000,0.000067


In [67]:
scaled_tvl

timestamp
2024-09-15 02:04:47+00:00    1.000000e-13
2024-09-15 05:44:35+00:00    1.000000e-13
2024-09-15 09:24:47+00:00    1.000000e-13
2024-09-15 13:04:11+00:00    1.000000e-13
2024-09-15 16:44:23+00:00    1.000000e-13
                                 ...     
2024-11-07 02:45:35+00:00    4.515054e+03
2024-11-07 06:25:23+00:00    4.515039e+03
2024-11-07 10:05:23+00:00    4.515158e+03
2024-11-07 13:45:47+00:00    4.497876e+03
2024-11-07 17:27:11+00:00    4.497834e+03
Length: 352, dtype: float64

In [63]:
scaled_nav_per_share

timestamp
2024-09-15 02:04:47+00:00    0.0
2024-09-15 05:44:35+00:00    0.0
2024-09-15 09:24:47+00:00    0.0
2024-09-15 13:04:11+00:00    0.0
2024-09-15 16:44:23+00:00    0.0
                            ... 
2024-11-07 02:45:35+00:00    0.0
2024-11-07 06:25:23+00:00    0.0
2024-11-07 10:05:23+00:00    0.0
2024-11-07 13:45:47+00:00    0.0
2024-11-07 17:27:11+00:00    0.0
Length: 352, dtype: float64

In [62]:
balETH_tvl_totalSupply

,getPoolsAndDestinations
timestamp,
2024-09-15 02:04:47+00:00,1.000000e-13
2024-09-15 05:44:35+00:00,1.000000e-13
2024-09-15 09:24:47+00:00,1.000000e-13
2024-09-15 13:04:11+00:00,1.000000e-13
2024-09-15 16:44:23+00:00,1.000000e-13
...,...
2024-11-07 02:45:35+00:00,4.493290e+03
2024-11-07 06:25:23+00:00,4.493282e+03
2024-11-07 10:05:23+00:00,4.493274e+03


In [58]:
scaled_tvl

timestamp
2024-09-15 02:04:47+00:00    1.000000e-13
2024-09-15 05:44:35+00:00    1.000000e-13
2024-09-15 09:24:47+00:00    1.000000e-13
2024-09-15 13:04:11+00:00    1.000000e-13
2024-09-15 16:44:23+00:00    1.000000e-13
                                 ...     
2024-11-07 02:45:35+00:00    4.515054e+03
2024-11-07 06:25:23+00:00    4.515039e+03
2024-11-07 10:05:23+00:00    4.515158e+03
2024-11-07 13:45:47+00:00    4.497876e+03
2024-11-07 17:27:11+00:00    4.497834e+03
Length: 352, dtype: float64

In [61]:
scaled_nav_per_share

timestamp
2024-09-15 02:04:47+00:00    0.0
2024-09-15 05:44:35+00:00    0.0
2024-09-15 09:24:47+00:00    0.0
2024-09-15 13:04:11+00:00    0.0
2024-09-15 16:44:23+00:00    0.0
                            ... 
2024-11-07 02:45:35+00:00    0.0
2024-11-07 06:25:23+00:00    0.0
2024-11-07 10:05:23+00:00    0.0
2024-11-07 13:45:47+00:00    0.0
2024-11-07 17:27:11+00:00    0.0
Length: 352, dtype: float64

In [60]:
import plotly.io as pio
pio.templates.default=None
px.line([tvl, scaled_tvl])

In [54]:
scaled_nav_per_share

timestamp
2024-09-15 02:04:47+00:00    0.0
2024-09-15 05:44:35+00:00    0.0
2024-09-15 09:24:47+00:00    0.0
2024-09-15 13:04:11+00:00    0.0
2024-09-15 16:44:23+00:00    0.0
                            ... 
2024-11-07 02:45:35+00:00    0.0
2024-11-07 06:25:23+00:00    0.0
2024-11-07 10:05:23+00:00    0.0
2024-11-07 13:45:47+00:00    0.0
2024-11-07 17:27:11+00:00    0.0
Length: 352, dtype: float64

In [53]:
nav_per_share_df = pd.DataFrame()
nav_per_share_df['balETH_actual_nav_per_share'] = balETH_nav_per_share
nav_per_share_df['balETH_scaled_nav_per_share'] = scaled_nav_per_share
px.line(nav_per_share_df)

In [ ]:
tvl # all these values are slightly off

timestamp
2024-09-17 08:02:11+00:00    668.513340
2024-09-17 08:02:23+00:00    668.472657
dtype: float64

In [39]:
balETH_tvl_totalAssets

,getPoolsAndDestinations
timestamp,
2024-09-17 08:02:11+00:00,668.504254
2024-09-17 08:02:23+00:00,668.456854


In [ ]:

# vaultAddress	: 0x003aD6a060a1082B7A6F85d24dDF9E9E196B8426
# exchangeName	: balancer
# dexPool	: 0xf01b0684C98CD7aDA480BFDF6e43876422fa1Fc1
# lpTokenAddress	: 0xf01b0684C98CD7aDA480BFDF6e43876422fa1Fc1
# lpTokenSymbol	: ECLP-wstETH-wETH
# lpTokenName	: Gyroscope ECLP wstETH/wETH
# autopool	: {'name': 'balETH', 'autopool_eth_addr': '0x6dC3ce9C57b20131347FDc9089D740DAf6eB34c5', 'autopool_eth_strategy_addr': '0xabe104560D0B390309bcF20b73Dca335457AA32e', 'solver_rebalance_plans_bucket': 'v2-gen3-eth-mainnet-solve-v2gen3ethmainnetsolverba-wx1snhldrupb'}
# vault_name	: ECLP-wstETH-wETH (balancer),th_addr': '0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56', 'autopool_eth_strategy_addr': '0xf5f6addB08c5e6091e5FdEc7326B21bEEd942235', 'solver_rebalance_plans_bucket': 'v2-gen3-eth-mainnet-solve-v2gen3ethmainnetsolverau-7ju0ra8aoznh'}


In [ ]:
call = build_proxyGetDestinationSummaryStats_call('Gyroscope ECLP wstETH/wETH', BAL_ETH.autopool_eth_strategy_addr, '0x003aD6a060a1082B7A6F85d24dDF9E9E196B8426', 'out', 0)

<Call proxyGetDestinationSummaryStats(address,address,uint8,uint256)((address,uint256,uint256,uint256,uint256,int256,int256,int256,uint256,int256,uint256)) on 0x146b55>